In [1]:
# -*- coding: utf-8 -*-
"""
Class definition of YOLO_v3 style detection model on image and video
"""

import colorsys

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, make_model
from yolo3.utils import letterbox_image
import os
from keras.utils import multi_gpu_model

class YOLO(object):
#     _defaults = {
#         "model_path": 'yolov3-tiny.h5',
#         "anchors_path": 'model_data/tiny_yolo_anchors.txt',
#         "classes_path": 'model_data/coco_classes.txt',
#         "score" : 0.3,
#         "iou" : 0.45,
#         "model_image_size" : (416, 416),
#         "text_size" : 3,
#         "gpu_num": 1,
#     }
    _defaults = {
        "model_path": 'yolo.h5',
        "anchors_path": 'model_data/yolo_anchors.txt',
        "classes_path": 'model_data/coco_classes.txt',
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "text_size" : 3,
        "gpu_num": 1,
    }
#     _defaults = {
#         "model_path": 'final_weight_1106.h5',
#         "anchors_path": 'model_data/yolo_anchors.txt',
#         "classes_path": 'coco_data/coco_classes.txt',
#         "score" : 0.10,
#         "iou" : 0.40,
#         "model_image_size" : (416, 416),
#         "text_size" : 3,
#         "gpu_num": 1,
#     }
    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
#         self.yolo_model = make_model(num_anchors//3, num_classes)
#         self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = make_model(num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'


        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        start = timer()

        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font='/usr/share/fonts/opentype/NotoSansCJK-Black.ttc', size=np.floor(3e-2 * image.size[1] + 0.1).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 500

        for i, c in reversed(list(enumerate(out_classes))):
            
            predicted_class = self.class_names[c]
#             if(predicted_class != 'car' and predicted_class != 'bus' and predicted_class!='truck'): continue
            box = out_boxes[i]
            score = out_scores[i]

#             label = '{} {:.2f}'.format(predicted_class, score)
            label = '{}'.format(predicted_class)
            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            print(label, (left, top), (right, bottom))

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=self.colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=self.colors[c])
            draw.text(text_origin, label, fill=(255, 255, 255), font=font)
            del draw

        end = timer()
        print(end - start)
        return image

    def close_session(self):
        self.sess.close()

def detect_video(yolo, video_path, output_path=""):
    import cv2
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
    video_fps       = vid.get(cv2.CAP_PROP_FPS)
    video_size      = (int(1920),
                        int(1080))
    isOutput = True if output_path != "" else False
    if isOutput:
        print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
        out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    while True:
        return_value, frame = vid.read()
        image = Image.fromarray(frame)
        image = yolo.detect_image(image)
        result = np.asarray(image)
        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1
        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0
        cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=1)
        cv2.namedWindow("result", cv2.WINDOW_NORMAL)
        cv2.imshow("result", result)
        if isOutput:
            out.write(result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    yolo.close_session()


Using TensorFlow backend.


In [2]:
    
yolo = YOLO()

Instructions for updating:
Colocations handled automatically by placer.
yolo.h5 model, anchors, and classes loaded.


In [3]:
detect_video(yolo, "HIGHWAY.mp4")


(416, 416, 3)
Found 0 boxes for img
4.4898985420004465
(416, 416, 3)
Found 0 boxes for img
0.05586383100126113
(416, 416, 3)
Found 0 boxes for img
0.05471131800004514
(416, 416, 3)
Found 0 boxes for img
0.054041822000726825
(416, 416, 3)
Found 0 boxes for img
0.0546872800005076
(416, 416, 3)
Found 0 boxes for img
0.06337329699999827
(416, 416, 3)
Found 0 boxes for img
0.05622511700130417
(416, 416, 3)
Found 0 boxes for img
0.0733027710011811
(416, 416, 3)
Found 0 boxes for img
0.06787451399941347
(416, 416, 3)
Found 0 boxes for img
0.06467858499854628
(416, 416, 3)
Found 0 boxes for img
0.061495674000980216
(416, 416, 3)
Found 0 boxes for img
0.06926373200076341
(416, 416, 3)
Found 0 boxes for img
0.06287116400017112
(416, 416, 3)
Found 0 boxes for img
0.060889604999829317
(416, 416, 3)
Found 0 boxes for img
0.06413342999985616
(416, 416, 3)
Found 0 boxes for img
0.06223557399971469
(416, 416, 3)
Found 0 boxes for img
0.058682666000095196
(416, 416, 3)
Found 0 boxes for img
0.059744144

Found 0 boxes for img
0.06002658000033989
(416, 416, 3)
Found 0 boxes for img
0.06284789399978763
(416, 416, 3)
Found 0 boxes for img
0.06401316099982068
(416, 416, 3)
Found 0 boxes for img
0.05878009400112205
(416, 416, 3)
Found 0 boxes for img
0.05940385899884859
(416, 416, 3)
Found 0 boxes for img
0.06318307399851619
(416, 416, 3)
Found 0 boxes for img
0.06239867600015714
(416, 416, 3)
Found 0 boxes for img
0.059565249999650405
(416, 416, 3)
Found 0 boxes for img
0.05930673299917544
(416, 416, 3)
Found 0 boxes for img
0.05846530800045002
(416, 416, 3)
Found 0 boxes for img
0.061127466999096214
(416, 416, 3)
Found 0 boxes for img
0.058781463998457184
(416, 416, 3)
Found 0 boxes for img
0.05865398599962646
(416, 416, 3)
Found 0 boxes for img
0.05541077099951508
(416, 416, 3)
Found 0 boxes for img
0.05897746600021492
(416, 416, 3)
Found 0 boxes for img
0.05949390600108018
(416, 416, 3)
Found 0 boxes for img
0.060147048001454095
(416, 416, 3)
Found 0 boxes for img
0.05748290500014264
(4

Found 0 boxes for img
0.06175234400143381
(416, 416, 3)
Found 0 boxes for img
0.06310539699916262
(416, 416, 3)
Found 0 boxes for img
0.0588685919992713
(416, 416, 3)
Found 0 boxes for img
0.05955545699907816
(416, 416, 3)
Found 0 boxes for img
0.061409846999595175
(416, 416, 3)
Found 0 boxes for img
0.06403075000162062
(416, 416, 3)
Found 0 boxes for img
0.06809454199901666
(416, 416, 3)
Found 0 boxes for img
0.06259884499922919
(416, 416, 3)
Found 0 boxes for img
0.06476934800048184
(416, 416, 3)
Found 0 boxes for img
0.06650230900049792
(416, 416, 3)
Found 0 boxes for img
0.06096235000040906
(416, 416, 3)
Found 0 boxes for img
0.06188211100015906
(416, 416, 3)
Found 0 boxes for img
0.06227256299825967
(416, 416, 3)
Found 0 boxes for img
0.061103855001420015
(416, 416, 3)
Found 0 boxes for img
0.060916037000424694
(416, 416, 3)
Found 0 boxes for img
0.06216668699926231
(416, 416, 3)
Found 1 boxes for img
bird (420, 247) (495, 310)
0.06218682800135866
(416, 416, 3)
Found 1 boxes for i

(416, 416, 3)
Found 5 boxes for img
car (644, 619) (659, 634)
car (472, 361) (490, 374)
car (601, 584) (613, 598)
car (599, 437) (615, 454)
car (643, 538) (661, 555)
0.07255023099969549
(416, 416, 3)
Found 5 boxes for img
car (607, 636) (618, 650)
car (643, 620) (660, 636)
car (599, 436) (616, 455)
car (602, 583) (615, 597)
car (642, 539) (661, 557)
0.07156163400031801
(416, 416, 3)
Found 4 boxes for img
car (607, 633) (618, 648)
car (603, 580) (616, 595)
car (598, 435) (616, 456)
car (644, 540) (661, 558)
0.06873654799892392
(416, 416, 3)
Found 4 boxes for img
car (593, 571) (605, 584)
car (603, 580) (616, 595)
car (598, 434) (615, 455)
car (644, 541) (661, 559)
0.07223249699927692
(416, 416, 3)
Found 6 boxes for img
car (607, 627) (618, 644)
car (642, 626) (658, 643)
car (593, 570) (605, 584)
car (604, 579) (617, 595)
car (598, 435) (615, 455)
car (644, 541) (661, 559)
0.072324700999161
(416, 416, 3)
Found 6 boxes for img
car (639, 468) (651, 481)
car (593, 570) (605, 583)
car (604, 

Found 6 boxes for img
truck (598, 412) (612, 431)
car (604, 567) (615, 582)
car (643, 517) (654, 527)
car (598, 412) (612, 431)
car (603, 529) (614, 544)
car (652, 593) (670, 611)
0.07447511899954407
(416, 416, 3)
Found 7 boxes for img
truck (598, 412) (612, 430)
car (653, 593) (669, 612)
car (604, 565) (615, 580)
car (642, 518) (655, 528)
car (598, 412) (612, 430)
car (599, 524) (611, 536)
car (603, 529) (615, 544)
0.0762659709998843
(416, 416, 3)
Found 7 boxes for img
truck (599, 412) (612, 430)
car (653, 600) (668, 622)
car (599, 412) (612, 430)
car (642, 519) (654, 529)
car (604, 529) (615, 545)
car (598, 523) (611, 536)
car (604, 564) (616, 578)
0.07287088299926836
(416, 416, 3)
Found 7 boxes for img
truck (599, 411) (611, 429)
car (599, 411) (611, 429)
car (641, 519) (654, 530)
car (603, 529) (615, 545)
car (604, 563) (615, 577)
car (653, 600) (669, 622)
car (597, 523) (612, 536)
0.07219473700024537
(416, 416, 3)
Found 6 boxes for img
truck (599, 411) (612, 429)
car (642, 520) (6

...

(416, 416, 3)
Found 13 boxes for img
truck (879, 545) (927, 574)
car (654, 520) (681, 552)
car (1029, 605) (1047, 621)
car (447, 600) (476, 622)
car (487, 517) (504, 531)
car (537, 459) (560, 480)
car (619, 436) (638, 459)
car (47, 558) (73, 574)
car (185, 635) (237, 659)
car (418, 611) (445, 629)
car (846, 643) (865, 659)
car (917, 639) (944, 658)
car (392, 518) (424, 535)
0.07876191600007587
(416, 416, 3)
Found 12 boxes for img
truck (879, 545) (927, 574)
car (305, 665) (342, 702)
car (1028, 604) (1047, 620)
car (620, 436) (637, 459)
car (538, 459) (560, 479)
car (487, 517) (504, 530)
car (47, 557) (72, 574)
car (187, 635) (238, 657)
car (418, 610) (445, 628)
car (917, 639) (944, 657)
car (846, 642) (866, 659)
car (392, 518) (425, 534)
0.07659679600146774
(416, 416, 3)
Found 13 boxes for img
truck (878, 545) (926, 574)
car (540, 460) (561, 479)
car (212, 635) (246, 654)
car (620, 435) (638, 458)
car (464, 591) (485, 609)
car (312, 660) (344, 688)
car (187, 634) (238, 654)
car (488, 5

car (909, 613) (932, 630)
car (649, 529) (674, 561)
car (240, 617) (273, 633)
car (911, 521) (932, 534)
car (875, 533) (921, 559)
car (859, 561) (878, 575)
car (492, 509) (507, 522)
car (1174, 678) (1213, 699)
car (464, 581) (492, 603)
car (52, 559) (70, 572)
car (542, 456) (562, 475)
car (841, 642) (860, 663)
car (1015, 587) (1035, 602)
car (321, 651) (357, 680)
0.08591355199860118
(416, 416, 3)
Found 16 boxes for img
truck (875, 534) (920, 559)
car (649, 529) (674, 561)
car (215, 613) (250, 632)
car (52, 560) (69, 573)
car (839, 601) (852, 616)
car (911, 521) (933, 534)
car (1175, 679) (1214, 698)
car (542, 457) (561, 474)
car (911, 614) (934, 631)
car (395, 507) (432, 526)
car (493, 508) (508, 521)
car (465, 581) (492, 601)
car (875, 534) (920, 559)
car (323, 650) (357, 679)
car (840, 641) (860, 663)
car (1015, 588) (1034, 602)
0.08186374399883789
(416, 416, 3)
Found 21 boxes for img
truck (832, 414) (855, 431)
truck (820, 411) (842, 429)
truck (874, 533) (920, 558)
car (910, 521) (

car (472, 566) (502, 586)
car (870, 526) (914, 549)
car (413, 497) (442, 512)
car (918, 612) (941, 626)
car (440, 581) (466, 599)
car (340, 627) (373, 657)
0.08283572899927094
(416, 416, 3)
Found 16 boxes for img
truck (812, 432) (846, 451)
truck (835, 411) (856, 426)
truck (640, 535) (665, 570)
truck (869, 525) (914, 548)
car (501, 497) (519, 510)
car (1181, 661) (1207, 680)
car (1179, 702) (1220, 720)
car (911, 518) (931, 531)
car (471, 566) (502, 585)
car (78, 540) (94, 552)
car (53, 557) (69, 569)
car (869, 525) (914, 548)
car (413, 497) (442, 512)
car (918, 611) (941, 625)
car (440, 580) (467, 598)
car (341, 627) (374, 657)
0.08683263700004318
(416, 416, 3)
Found 17 boxes for img
truck (640, 536) (666, 570)
truck (812, 432) (846, 451)
truck (835, 411) (855, 426)
truck (868, 524) (914, 548)
car (1009, 567) (1024, 580)
car (241, 603) (277, 621)
car (1179, 701) (1216, 720)
car (1181, 661) (1207, 680)
car (263, 604) (297, 622)
car (471, 567) (502, 585)
car (53, 557) (69, 569)
car (868

Found 18 boxes for img
truck (874, 511) (912, 533)
truck (819, 410) (838, 424)
truck (635, 546) (659, 585)
truck (1167, 652) (1215, 681)
car (549, 443) (568, 456)
car (874, 511) (912, 533)
car (445, 564) (471, 582)
car (505, 492) (521, 504)
car (719, 545) (739, 558)
car (848, 590) (862, 604)
car (385, 653) (416, 688)
car (795, 643) (818, 661)
car (262, 584) (304, 604)
car (286, 586) (320, 605)
car (993, 562) (1011, 576)
car (418, 490) (450, 507)
car (1157, 682) (1194, 706)
car (356, 609) (389, 637)
0.09277490999920701
(416, 416, 3)
Found 20 boxes for img
truck (861, 512) (904, 537)
truck (819, 410) (838, 424)
truck (1165, 651) (1214, 679)
truck (634, 547) (658, 586)
car (1165, 651) (1214, 679)
car (917, 595) (940, 609)
car (848, 590) (862, 605)
car (263, 584) (301, 603)
car (505, 492) (521, 504)
car (76, 540) (92, 552)
car (718, 546) (739, 558)
car (445, 564) (471, 581)
car (470, 559) (499, 579)
car (794, 642) (816, 661)
car (385, 652) (417, 688)
car (287, 585) (320, 605)
car (1140, 65

(416, 416, 3)
Found 15 boxes for img
truck (819, 396) (832, 409)
truck (1121, 657) (1156, 677)
truck (1102, 627) (1144, 648)
truck (620, 560) (648, 597)
truck (858, 503) (895, 526)
car (513, 484) (529, 496)
car (986, 539) (1000, 552)
car (651, 481) (665, 494)
car (917, 583) (941, 597)
car (287, 565) (337, 587)
car (889, 496) (909, 507)
car (462, 552) (487, 570)
car (492, 542) (515, 559)
car (1121, 657) (1156, 677)
car (371, 588) (404, 616)
0.0828932349995739
(416, 416, 3)
Found 12 boxes for img
truck (819, 396) (832, 409)
truck (1100, 626) (1143, 647)
truck (617, 569) (644, 605)
truck (858, 502) (896, 526)
car (513, 484) (529, 496)
car (917, 582) (941, 596)
car (463, 552) (487, 569)
car (889, 495) (908, 507)
car (287, 564) (336, 587)
car (1120, 656) (1155, 676)
car (492, 541) (516, 559)
car (372, 587) (404, 615)
0.07993646499926399
(416, 416, 3)
Found 13 boxes for img
truck (1119, 654) (1156, 676)
truck (889, 494) (908, 507)
truck (818, 395) (832, 409)
truck (1096, 624) (1140, 646)
tru